# 5.0 Going Modular: Part 1 (Cell Mode)

This notebook is a cleaned up version of notebook5 TinyVGG trained on images of pizza, steak and sushi.

This will be used as the base to make a script version of the same code to bring it modular

## 1. Get Data

Download the same data from notebook 5, the `pizza_steak_sushi` dataset images of pizza, steak and sushi

In [17]:
import os
import zipfile
from pathlib import Path
import requests

data_path = Path("data/")
image_path = data_path / 'pizza_steak_sushi'

# Make folder and download the data
if image_path.is_dir():
  print(f'{image_path} already exists!')
else:
  print(f'Did not find {image_path}, making directory')
  image_path.mkdir(parents=True, exist_ok=True)

with open(data_path / 'pizza_steak_sushi.zip' , 'wb') as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("Downloading data.....")
  f.write(request.content)

with zipfile.ZipFile(data_path/'pizza_steak_sushi.zip', 'r') as zip_ref:
  print('Unzipping pizza, steak, sushi data ....')
  zip_ref.extractall(image_path)

os.remove(data_path / 'pizza_steak_sushi.zip')

data\pizza_steak_sushi already exists!
Unzipping pizza, steak, sushi data ....


In [18]:
train_dir = image_path / 'train'
test_dir = image_path / 'test'

train_dir, test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

## 2. Create Datasets and DataLoaders

Cast images into datasets, then into dataloaders

In [19]:
from torchvision import datasets, transforms

# Simple transform
data_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])

# Using ImageFolder to create Datsets
train_data = datasets.ImageFolder(root=train_dir, transform=data_transform, target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, target_transform=None, transform=data_transform)

print(f'Train data: \n{train_data}\nTest data\n{test_data}')

Train data: 
Dataset ImageFolder
    Number of datapoints: 225
    Root location: data\pizza_steak_sushi\train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Test data
Dataset ImageFolder
    Number of datapoints: 75
    Root location: data\pizza_steak_sushi\test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [20]:
class_names = train_data.classes
class_names

['pizza', 'steak', 'sushi']

In [21]:
class_dict = train_data.class_to_idx
class_dict

{'pizza': 0, 'steak': 1, 'sushi': 2}

In [22]:
len(train_data), len(test_data)

(225, 75)

In [23]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset=train_data, batch_size=1, num_workers=1, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=1, num_workers=1, shuffle=False)

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x1f67a8b52b0>,
 <torch.utils.data.dataloader.DataLoader at 0x1f67a8b5af0>)

In [24]:
img, label = next(iter(train_dataloader))

print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f'Label shape: {label.shape}')

Image shape: torch.Size([1, 3, 64, 64]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([1])


### 2.1 Create Datasets adn Dataloaders (script mode)

Let us use the Jupyter magic command to create a `.py` file

Save the code cell's contents to a file using the Jupyter magics `%%writefile filename`m

In [25]:
# Create a directory
import os
os.makedirs('going_modular')

In [26]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoader's for image classification data.
"""
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

num_workers = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int = num_workers
):
  """
  Creates training and testing DataLoaders.
  Takes in a training and testing directory path and turns them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader

  Returns:
  A tuple of (train_dataloader, test_dataloader, class_names).
  Where class_names is a list of the target classes.

  Example usage:
    train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=path/to/train_dir, test_dir=path/to/test_dir, transform=some_transform, batch_size=32, num_workers=4)
  """

  print('Creating Datasets........')
  train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
  test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
  print(f'Train and Test datasets created')

  print('Creating DataLoaders........')
  train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
  test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, pin_memory=True)
  print(f'Train and Test DataLoaders created')

  class_names = train_dataset.classes

  return train_dataloader, test_dataloader, class_names

Writing going_modular/data_setup.py


In [27]:
import importlib
from going_modular import data_setup
importlib.reload(data_setup)

# Now use it
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=str(train_dir),
    test_dir=str(test_dir),
    batch_size=32,
    transform=data_transform
)

Creating Datasets........
Train and Test datasets created
Creating DataLoaders........
Train and Test DataLoaders created


## 3. Making a model (TinyVGG)

TinyVGG from the CNN Explainer Website with docstring

In [28]:
import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates a TinyVGG architecture.

  Replicates the TinyVGG acrchitecture from the CNN explainer website in Pytorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels,
    hidden_units: An integer indicating number of hidden units between layers,
    output_shape: An integer indicating number of outptu units

  """
  def __init__(self, input_shape: int, hidden_units:int, output_shape:int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        # input_featuer len comes form hidden units being compressed
        nn.Linear(in_features=hidden_units*13*13, out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

In [29]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(42)
model_0 = TinyVGG(input_shape=3, hidden_units=10, output_shape=len(train_data.classes)).to(device)
model_0

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

Dummy forward pass

In [30]:
img_batch, label_batch = next(iter(train_dataloader))

img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

model_0.eval()
with torch.inference_mode():
  pred = model_0(img_single.to(device))

print(f'Output logits: \n{pred}\n')
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits: 
tensor([[ 0.0208, -0.0020,  0.0095]])

Output prediction probabilities:
tensor([[0.3371, 0.3295, 0.3333]])

Output prediction label:
tensor([0])

Actual label:
0


## 3.1 Making a model (TinyVGG) wiht a script (`model_builder.py`)

In [31]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model,
from the CNN explainer website
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates a TinyVGG architecture.

  Replicates the TinyVGG acrchitecture from the CNN explainer website in Pytorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels,
    hidden_units: An integer indicating number of hidden units between layers,
    output_shape: An integer indicating number of outptu units

  """
  def __init__(self, input_shape: int, hidden_units:int, output_shape:int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        # input_featuer len comes form hidden units being compressed
        nn.Linear(in_features=hidden_units*13*13, out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

Writing going_modular/model_builder.py


In [32]:
from going_modular import model_builder

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(42)
model_1 = model_builder.TinyVGG(3,10,len(class_names)).to(device)
model_1

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [33]:
img_batch, label_batch = next(iter(train_dataloader))

img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

model_1.eval()
with torch.inference_mode():
  pred = model_0(img_single.to(device))

print(f'Output logits: \n{pred}\n')
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits: 
tensor([[ 0.0208, -0.0020,  0.0095]])

Output prediction probabilities:
tensor([[0.3371, 0.3295, 0.3333]])

Output prediction label:
tensor([0])

Actual label:
0


## 4. Creating `train_step()` and `test_step()` functions and combine them for `train()`

In [34]:
from typing import Tuple

def train_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer, device: torch.device) -> Tuple[float, float]:
  """ Trains a PyTorch model for a single epoch.

  Truns a target PyTorch model to training mode and then runs through all the required
  training steps (forward pass, loss calculation, optimizer step).

  Args:
    mode: A PyTorch model to be trained.
    dataLoader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorcch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. 'cuda' or 'cpu').

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:
    (0.112, 0.8743)
 """
  model.train()

  train_loss = train_acc = 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  train_loss = train_loss/ len(dataloader)
  train_acc = train_acc/ len(dataloader)
  return train_loss, train_acc


In [35]:
def test_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, device: torch.device) -> Tuple[float, float]:
  """ Tests a PyTorch model for a single epoch.

  Truns a target PyTorch model to eval mode and then runs forward pass.

  Args:
    mode: A PyTorch model to be trained.
    dataLoader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    device: A target device to compute on (e.g. 'cuda' or 'cpu').

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:
    (0.112, 0.8743)
 """
  model.train()

  test_loss = test_acc = 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    test_pred_logits = model(X)
    loss = loss_fn(test_pred_logits, y)
    test_loss += loss.item()
    test_pred_labels = test_pred_logits.argmax(dim=1)
    test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

In [36]:
from typing import Dict, List
from tqdm.auto import tqdm

def train(model: torch.nn.Module, train_dataloader: torch.utils.data.DataLoader, test_dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer, device: torch.device, epochs: int) -> Dict[str, List[float]]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  results = {'train_loss': [],
             'train_acc': [],
             'test_loss': [],
             'test_acc': []
             }

  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model, dataloader=train_dataloader, loss_fn=loss_fn, device=device, optimizer=optimizer)
    test_loss, test_acc = test_step(model=model, loss_fn=loss_fn, dataloader=test_dataloader, device=device)

    print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

    results['train_loss'].append(train_loss)
    results['test_loss'].append(test_loss)
    results['train_acc'].append(train_acc)
    results['test_acc'].append(test_acc)

  return results

## 4.1 Training functions into a script (`engine.py`)

In [37]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing PyTorch model.
"""
from typing import Dict, List, Tuple
import torch
from tqdm.auto import tqdm

def train_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer, device: torch.device) -> Tuple[float, float]:
  """ Trains a PyTorch model for a single epoch.

  Truns a target PyTorch model to training mode and then runs through all the required
  training steps (forward pass, loss calculation, optimizer step).

  Args:
    mode: A PyTorch model to be trained.
    dataLoader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorcch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. 'cuda' or 'cpu').

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:
    (0.112, 0.8743)
 """
  model.train()

  train_loss = train_acc = 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  train_loss = train_loss/ len(dataloader)
  train_acc = train_acc/ len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, device: torch.device) -> Tuple[float, float]:
  """ Tests a PyTorch model for a single epoch.

  Truns a target PyTorch model to eval mode and then runs forward pass.

  Args:
    mode: A PyTorch model to be trained.
    dataLoader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    device: A target device to compute on (e.g. 'cuda' or 'cpu').

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:
    (0.112, 0.8743)
 """
  model.eval()

  test_loss = test_acc = 0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      test_pred_logits = model(X)
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

from typing import Dict, List
from tqdm.auto import tqdm

def train(model: torch.nn.Module, train_dataloader: torch.utils.data.DataLoader, test_dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer, device: torch.device, epochs: int) -> Dict[str, List[float]]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  results = {'train_loss': [],
             'train_acc': [],
             'test_loss': [],
             'test_acc': []
             }

  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model, dataloader=train_dataloader, loss_fn=loss_fn, device=device, optimizer=optimizer)
    test_loss, test_acc = test_step(model=model, loss_fn=loss_fn, dataloader=test_dataloader, device=device)

    print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

    results['train_loss'].append(train_loss)
    results['test_loss'].append(test_loss)
    results['train_acc'].append(train_acc)
    results['test_acc'].append(test_acc)

  return results


Writing going_modular/engine.py


In [38]:
from going_modular import engine

engine.train(train_dataloader=train_dataloader,
             model=model_1,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             device=device,
             epochs=10)

NameError: name 'loss_fn' is not defined

## 5. Creating a function to save the model

In [39]:
def save_model(model:torch.nn.Module, target_dir:str, model_name:str):
  """ Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should Include either ".pth' or ".pt"

  Example usage:
    save_model(model=model_0,
                target_dir="models",
                model_name="05_going_modular_tingvgg_model.pth")
  """
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(exist_ok=True, parents=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith('.pt'), 'model_name should end with .pth or .pt'
  model_save_path = target_dir_path/model_name

  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(), f=model_save_path)

## 5.1 File to save utility functions called `utility.py`

In [40]:
%%writefile going_modular/utils.py
"""
Contains function to save the PyTorch model
"""
import torch
from pathlib import Path

def save_model(model:torch.nn.Module, target_dir:str, model_name:str):
  """ Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should Include either ".pth' or ".pt"

  Example usage:
    save_model(model=model_0,
                target_dir="models",
                model_name="05_going_modular_tingvgg_model.pth")
  """
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(exist_ok=True, parents=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith('.pt'), 'model_name should end with .pth or .pt'
  model_save_path = target_dir_path/model_name

  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(), f=model_save_path)

Writing going_modular/utils.py


## 6. Train, evaluate and save the model

In [41]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

NUM_EPOCHS = 10

model_0 = TinyVGG(
    input_shape=3,
    hidden_units=10,
    output_shape=len(train_data.classes)
).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

from timeit import default_timer as timer

start_time = timer()

model_0_results = train(model=model_0,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS,
                        device=device)

end_time = timer()

print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

# Save the model
save_model(model=model_0,
           target_dir="models",
           model_name="05_going_modular_cell_mode_tinyvgg_model.pth")

 10%|█         | 1/10 [00:29<04:24, 29.39s/it]

Epoch: 1 | train_loss: 1.1063 | train_acc: 0.3047 | test_loss: 1.0983 | test_acc: 0.3011


 20%|██        | 2/10 [00:56<03:42, 27.78s/it]

Epoch: 2 | train_loss: 1.0998 | train_acc: 0.3281 | test_loss: 1.0697 | test_acc: 0.5417


 30%|███       | 3/10 [01:22<03:09, 27.03s/it]

...

Epoch: 8 | train_loss: 0.9607 | train_acc: 0.5820 | test_loss: 1.0057 | test_acc: 0.4640


 90%|█████████ | 9/10 [04:44<00:31, 31.89s/it]

Epoch: 9 | train_loss: 0.9260 | train_acc: 0.5938 | test_loss: 1.0732 | test_acc: 0.3324


100%|██████████| 10/10 [05:15<00:00, 31.51s/it]

Epoch: 10 | train_loss: 1.0013 | train_acc: 0.4570 | test_loss: 1.0571 | test_acc: 0.4044
[INFO] Total training time: 315.130 seconds
[INFO] Saving model to: models\05_going_modular_cell_mode_tinyvgg_model.pth


## 6.1 Train, evaluate and save mode (script mode)  `train.py`

The master script uses all the other scrpits that were made above

In [42]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""
import torch
import os
from torchvision import transforms
import data_setup
import utils
import engine
import model_builder
from timeit import default_timer as timer

# Set seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Setup Hyper parameters
learning_rate = 0.001
num_epochs = 3
batch_size = 32
hidden_units = 10
input_shape = 3


# Setup directories
train_dir = 'data/pizza_steak_sushi/train'
test_dir = 'data/pizza_steak_sushi/test'

# Device agnostice code
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# transformmation
data_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

# Create dataloaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                            test_dir=test_dir,
                                                            transform=data_transform,
                                                            batch_size=batch_size)

# Create model
model = model_builder.TinyVGG(input_shape=3, hidden_units=10, output_shape=len(class_names))

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

start_time = timer()

# Train model with engine
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             device=device,
             epochs=num_epochs
             )

end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

# Save the model using utils
utils.save_model(model=model,
                 model_name='06_going_modular_script_mode.pth',
                 target_dir='models')

Writing going_modular/train.py


In [ ]:
!python going_modular/train.py